### 加载模型

In [1]:
from detection.DataGenerator import train_based_self_detection
from detection.Loader.ResNet50Loader import Resnet50Loader
import detection.Spliter
import torch
from torch import nn
print("CODE:loading_resnet50")
model=Resnet50Loader().load()
print("CODE:loading_finished")
device='cuda:3'
back_device='cuda:3'
quantisized_type=torch.qint8
cut_step=0.2

CODE:loading_resnet50


/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CODE:loading_finished


### 创建量化模型用于测试acc

In [2]:
from torch.quantization.observer import MovingAveragePerChannelMinMaxObserver
observer = MovingAveragePerChannelMinMaxObserver(ch_axis=0,dtype=quantisized_type).to(device)

class quantiseze_model(nn.Module):
    def __init__(self,model_list):
        super(quantiseze_model,self).__init__()
        self.model_list=model_list
    def forward(self,x):
        x=self.model_list[0](x)
        observer(x)
        scale, zero_point = observer.calculate_qparams()
        # scale=scale.to(device)
        # zero_point=zero_point.to(device)
        x_quantized = torch.quantize_per_channel(x, scales=scale, zero_points=zero_point, axis=0, dtype=quantisized_type)
        x=self.model_list[1](x_quantized.dequantize())
        observer(x)
        scale, zero_point = observer.calculate_qparams()
        # scale=scale.to(device)
        # zero_point=zero_point.to(device)
        x_quantized = torch.quantize_per_channel(x, scales=scale, zero_points=zero_point, axis=0, dtype=quantisized_type)
        x=self.model_list[2](x_quantized.dequantize())
        return x

### 创建测试数据

In [3]:
datamaker=train_based_self_detection(
    model=model,
    device=device,
    no_weight=True
)

input_data,output_label,label,highest_loss,lowest_loss= datamaker.make_data_pid(
        total_number=100,
        batch_size=100,
        learning_rate=1,
        warm_lr=1e-3,
        channel=3,
        dim1=224,
        dim2=224,
        output_size=1000,
        randn_magnification=100,
        confidence=1000000,
        target_acc=0.8

)
print(input_data.dtype)
torch.cuda.empty_cache()

/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.

acc: 0.82 loss: 1.4887244701385498
torch.float32


### 初始化搜索器

In [ ]:

searcher=detection.Spliter.Recrusively_reduce_search(
        model=model,
        no_weight=True,
        input_data=input_data,
        output_label=output_label,
        label=label,
        device=device,
        back_device=back_device,
        highest_loss=highest_loss,
        lowest_loss=lowest_loss,
        local_speed=2.72e10,   #Flops/s
        # local_speed=9.6e9,   #Flops/s
        cloud_speed=1.7e13,    #Flops/s
        network_speed=1e7,     #B/s
        acc_cut_point=0.7,
        # q=q,
)
searcher.init(cut_step)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_op

0it [00:00, ?it/s]

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() f

9it [00:00, 18.09it/s]
7it [00:00, 38.88it/s]
7it [00:00, 43.93it/s]
9it [00:00, 27.22it/s]
7it [00:00, 36.72it/s]
7it [00:00, 33.59it/s]
7it [00:00, 16.83it/s]]
9it [00:00, 13.11it/s]]
7it [00:00, 13.35it/s]]
7it [00:00, 19.77it/s]]
7it [00:00, 22.09it/s]]
7it [00:00, 14.07it/s]]
7it [00:00, 22.36it/s]]
9it [00:01,  6.29it/s]]
7it [00:00,  9.64it/s]]
7it [00:00,  9.07it/s]]
20it [00:07,  1.36it/s]/home/tianruiming/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
23it [00:08,  2.68it/s]
23it [00:00, 115.60it/s]


SVD_finished
Loss上限: 6.219280242919922
acc上限: 0.0
最长耗时: 17.57764629684706
最短耗时: 9.939687708611764


quantized_acc_list: [0.84, 0.81, 0.82, 0.81, 0.81, 0.81, 0.81, 0.81, 0.82, 0.82, 0.82, 0.84, 0.82, 0.83, 0.84, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.84, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.84, 0.84, 0.84, 0.84, 0.85, 0.85]
quantized_network_list: [8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 8.02816, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 4.01408, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 2.00704, 1.00352, 1.00352, 1.00352, 1.00352, 1.00352, 1.00352]
quantized_compute_list: [0.4031120037647059, 0.45924064828235295, 0.7470147659294117, 0.8909018247529412, 1.03478888357

In [5]:
searcher.input_data.shape
upper_bound=searcher.GA_init(50,step=cut_step)

### 计算每个分割点的时间和准确率

In [6]:
print()
upper_num=min(upper_bound)
quantized_network_list=[]
quantized_compute_list=[]
quantized_time_list=[]
quantized_acc_list=[]
for i in range(1,50):
    torch.cuda.empty_cache()
    cut_num=int((max(upper_bound[:i+1])+min(upper_bound[:i+1]))/2)
    model_r,edge_layer_map=searcher.model_reduce([cut_num]*i)
    model_nr,edge_layer_map=searcher.model_reduce([0]*i)
    eA_r,c_r,eB_r=searcher.split(model_r,len(edge_layer_map))
    # print(model_r)
    # input()
    eA_nr,c_nr,eB_nr=searcher.split(model_nr,len(edge_layer_map))
    qm_r=quantiseze_model([eA_r,c_r,eB_r])
    qm_r.eval()
    qm_nr=quantiseze_model([eA_nr,c_nr,eB_nr])
    qm_nr.eval()
    quantized_network_list.append(searcher.network_evaluate_quantisized(qm_r,quantisized_type))
    acc,loss=searcher.acc_loss_evaluate(qm_nr)
    compute_time=searcher.latency_evaluate(eA_r,c_r,eB_r)
    quantized_compute_list.append(compute_time)
    quantized_acc_list.append(acc)
    # quantized_time_list.append(compute_time+searcher.network_evaluate_quantisized(qm_r))
    quantized_time_list.append(quantized_network_list[-1]+quantized_compute_list[-1])
    # input()
    torch.cuda.empty_cache()

In [7]:
print()
print("quantized_acc_list:",quantized_acc_list)
print("quantized_network_list:",quantized_network_list)
print("quantized_compute_list:",quantized_compute_list)
print("quantized_time_list:",quantized_time_list)

### 归一化分割点时间和准确率

In [8]:
print()
print("min_quantized_time_list:",min(quantized_time_list))
print("max_quantized_time_list:",max(quantized_time_list))
normaled_time=[(x-min(quantized_time_list))/(max(quantized_time_list)-min(quantized_time_list)) for x in quantized_time_list]
normaled_acc=[(x-min(quantized_acc_list))/(max(quantized_acc_list)-min(quantized_acc_list)) for x in quantized_acc_list]
print("normaled_time:",normaled_time)
print("normaled_acc:",normaled_acc)

### 计算各点"伪"F得分

In [9]:
import numpy as np
alpha=0.5
print()
def F_score(alpha,index):
    return float(alpha*np.exp(1-normaled_time[index])+(1-alpha)*np.exp(normaled_acc[index]))
F_per_point=[F_score(alpha,i) for i in range(0,48)]


for i in range(0,48):
    print("idx:",i,"net_time:",quantized_network_list[i],"compute_time:",quantized_compute_list[i],"acc:",quantized_acc_list[i],"F_per_point:",F_per_point[i])
F_per_point=[0,0]+F_per_point

### 确定最优划分点

In [10]:
best_index=F_per_point.index(max(F_per_point))
best_index
torch.cuda.empty_cache()

### 依据最优划分点进行搜索

In [11]:
searcher.search_GA(
    number_of_layer_to_reduce=best_index,
    alpha=0.7,
    step=cut_step
)

RuntimeError: Cowardly refusing to serialize non-leaf tensor which requires_grad, since autograd does not support crossing process boundaries.  If you just want to transfer the data, call detach() on the tensor before serializing (e.g., putting it on the queue).